In [1]:
import torch
import tqdm
from config import *
from utils import *
import os
import numpy as np
import shutil
import matplotlib.pyplot as plt
import articulate as art
from articulate.utils.rbdl import *
from net import PIP

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
net = PIP()
net.to(device)

PIP(
  (rnn1): RNNWithInit(
    (rnn): LSTM(256, 256, num_layers=2, dropout=0.4)
    (linear1): Linear(in_features=72, out_features=256, bias=True)
    (linear2): Linear(in_features=256, out_features=15, bias=True)
    (dropout): Dropout(p=0.4, inplace=False)
    (init_net): Sequential(
      (0): Linear(in_features=15, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=512, bias=True)
      (3): ReLU()
      (4): Linear(in_features=512, out_features=1024, bias=True)
    )
  )
  (rnn2): RNN(
    (rnn): LSTM(256, 256, num_layers=2, dropout=0.4)
    (linear1): Linear(in_features=87, out_features=256, bias=True)
    (linear2): Linear(in_features=256, out_features=69, bias=True)
    (dropout): Dropout(p=0.4, inplace=False)
  )
  (rnn3): RNN(
    (rnn): LSTM(256, 256, num_layers=2, dropout=0.4)
    (linear1): Linear(in_features=141, out_features=256, bias=True)
    (linear2): Linear(in_features=256, out_features=90, bias=True)
    (dropout): Dropo

In [4]:
data_dir = paths.totalcapture_dir

data_name = os.path.basename(data_dir)

data_dir, data_name

('data/dataset_work/TotalCapture', 'TotalCapture')

In [5]:
acc_all, rot_all, pose_all, tran_all = torch.load(os.path.join(data_dir, 'test.pt')).values()
init_poses = [art.math.axis_angle_to_rotation_matrix(_[0]) for _ in pose_all]

sequence_ids = list(range(len(acc_all)))

In [6]:
len(acc_all), len(rot_all), len(pose_all), len(tran_all), len(init_poses)

(41, 41, 41, 45, 41)

In [7]:
    for i in tqdm.tqdm(sequence_ids):
        acc_t, rot_t = acc_all[i], rot_all[i]
        pose_t, tran_t = pose_all[i], tran_all[i]
        init_pose_t = init_poses[i]
        print(acc_t.shape, rot_t.shape)
        print(pose_t.shape, tran_t.shape)
        print(init_pose_t.shape)
        break

  0%|          | 0/41 [00:00<?, ?it/s]

torch.Size([4113, 6, 3]) torch.Size([4113, 6, 3, 3])
torch.Size([4113, 24, 3]) torch.Size([4113, 3])
torch.Size([24, 3, 3])


In [8]:
## config.py setting
# joint_set
# vel_scale

In [9]:
# x, lj_init, jvel_init = list(zip(*[x]))

# x[0].shape, lj_init[0].shape, jvel_init[0].shape
# x[0].dtype, lj_init[0].dtype, jvel_init[0].dtype

In [13]:
        glb_acc, glb_rot, init_pose = acc_t.to(device), rot_t.to(device), init_pose_t.to(device)
        
        # init pose
        init_pose = init_pose.view(1, 24, 3, 3)
        init_pose[0, 0] = torch.eye(3)
        # global pose
        pose_global, joint_global = net.forward_kinematics(init_pose) # calc_mesh == False
        # leaf(1 order) joint
        lj_init = joint_global[0, joint_set.leaf].view(-1)
        # init velocity
        jvel_init = torch.zeros(24 * 3)
        # init all joint pose x
        # x = (normalize_and_concat(glb_acc, glb_rot), lj_init, jvel_init) # x = (frames, 72)
        x = (normalize_and_concat(glb_acc, glb_rot).to(device), 
             lj_init.to(device), 
             jvel_init.to(device)) # x = (frames, 72)
        # Neural Kinematic Estimator
        leaf_joint, full_joint, global_6d_pose, joint_velocity, contact = [_[0] for _ in net.forward([x])]
        pose = net._reduced_glb_6d_to_full_local_mat(glb_rot.view(-1, 6, 3, 3)[:, -1], global_6d_pose)
        joint_velocity = joint_velocity.view(-1, 24, 3).bmm(glb_rot[:, -1].transpose(1, 2)) * vel_scale
        
        # compute loss
        
        # detach grad
        pose = pose.cpu().detach()
        joint_velocity = joint_velocity.cpu().detach()
        contact = contact.cpu().detach()
        glb_acc = glb_acc.cpu().detach()
        
        # Physics-aware Motion Optimizer
        pose_opt, tran_opt = [], []
        for p, v, c, a in zip(pose, joint_velocity, contact, glb_acc):
            p, t = net.dynamics_optimizer.optimize_frame(p, v, c, a)
            pose_opt.append(p)
            tran_opt.append(t)
        pose_opt, tran_opt = torch.stack(pose_opt), torch.stack(tran_opt)

In [ ]:
# clip 200 frames
# batch 256
# Adam optimizer
# fine-tune PL (with IPL), PA, and RA on the train split of the DIP-IMU 
# do not train VA and CF on DIP-IMU (does not contain global movements)

In [ ]:
def loss_function():
    L_PL = torch.nn.functional.mse_loss(leaf_joint, ) # sec.3.1.1
    L_PA = torch.nn.functional.mse_loss(full_joint, ) # sec.3.1.1
    L_RA = torch.nn.functional.mse_loss(global_6d_pose, ) # sec.3.1.1
    
    L_CF = torch.nn.functional.binary_cross_entropy(contact, ) # sec.3.1.1
    
    L_VA = torch.nn.functional.(joint_velocity, ) # [73]:TransPose eq.(7)

In [ ]:
joint_velocity_GT = ()() / delta_t # sec.A eq.(14)